In [1]:
import h2o
from bayes_opt import BayesianOptimization

In [2]:
h2o.init()
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,17 days 1 hour 34 mins
H2O cluster timezone:,America/Santiago
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.3
H2O cluster version age:,"28 days, 19 hours and 20 minutes"
H2O cluster name:,H2O_from_python_maravenag_3wzuvj
H2O cluster total nodes:,1
H2O cluster free memory:,3.268 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [3]:
data = h2o.upload_file("data/winequality-red.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
data.head(10)

fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8,5
7.8,0.76,0.04,2.3,0.092,15,54,0.997,3.26,0.65,9.8,5
11.2,0.28,0.56,1.9,0.075,17,60,0.998,3.16,0.58,9.8,6
7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
7.4,0.66,0,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5
7.9,0.6,0.06,1.6,0.069,15,59,0.9964,3.3,0.46,9.4,5
7.3,0.65,0,1.2,0.065,15,21,0.9946,3.39,0.47,10,7
7.8,0.58,0.02,2,0.073,9,18,0.9968,3.36,0.57,9.5,7
7.5,0.5,0.36,6.1,0.071,17,102,0.9978,3.35,0.8,10.5,5


In [5]:
train_cols = [x for x in data.col_names if x not in ['quality'] ] 

In [6]:
train, test = data.split_frame(ratios=[0.7])

In [7]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [8]:
def train_model(max_depth, 
                ntrees,
                min_rows, 
                learn_rate, 
                sample_rate, 
                col_sample_rate):
    params = {
        'max_depth': int(max_depth),
        'ntrees': int(ntrees),
        'min_rows': int(min_rows),
        'learn_rate':learn_rate,
        'sample_rate':sample_rate,
        'col_sample_rate':col_sample_rate
    }
    model = H2OGradientBoostingEstimator(nfolds=5,**params)
    model.train(x=train_cols, y='quality', training_frame=train)
    return -model.rmse()

In [9]:
bounds = {
    'max_depth':(5,10),
    'ntrees': (100,500),
    'min_rows':(10,30),
    'learn_rate':(0.001, 0.01),
    'sample_rate':(0.5,0.8),
    'col_sample_rate':(0.5,0.8)
}

In [10]:
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)

In [11]:
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | col_sa... | learn_... | max_depth | min_rows  |  ntrees   | sample... |
-------------------------------------------------------------------------------------------------
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  1        | -0.5997   |  0.6251   |  0.007483 |  5.001    |  16.05    |  158.7    |  0.5277   |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  2        | -0.5934   |  0.5559   |  0.00411  |  6.984    |  20.78    |  267.7    |  0.7056   |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  3        | -0.5472   |  0.5613   |  0.008903 |  5.137    |  23.41    |  266.9    |  0.6676   |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  4        | -0.6594   |  0.5421   |  0.002783 |  9.004    |  29.37    |  225.4    |  0.7077   |
gbm Model Build progress: |███████████████████████████████████████████████| 100%
|  5   

In [12]:
optimizer.max

{'target': -0.28765789542728204,
 'params': {'col_sample_rate': 0.8,
  'learn_rate': 0.01,
  'max_depth': 10.0,
  'min_rows': 10.0,
  'ntrees': 500.0,
  'sample_rate': 0.8}}